In [12]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.spatial.distance import correlation
import glob

In [13]:
def calculate_cohens_d(group1, group2):
    n1, n2 = len(group1), len(group2)
    var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
    
    # Pooled standard deviation
    pooled_se = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
    
    # Cohen's d
    d = (np.mean(group1) - np.mean(group2)) / pooled_se
    return d

In [14]:
def calculate_hedges_g(group1, group2):
    # Calculate Cohen's d first
    d = calculate_cohens_d(group1, group2)
    
    # Calculate correction factor
    n1, n2 = len(group1), len(group2)
    N = n1 + n2
    correction = 1 - (3 / (4 * (N - 2) - 1))
    
    # Calculate Hedges' g
    g = correction * d
    return g

In [15]:
def analyze_correlations(gt_data, disease_data, case_type="confirmed", data_type="MSV"):
    # Convert all dates to datetime
    gt_data = gt_data.copy()
    disease_data = disease_data.copy()
    gt_data['date'] = pd.to_datetime(gt_data['date'])
    disease_data['date'] = pd.to_datetime(disease_data['date'])
    
    # Set date as index for both dataframes
    gt_data.set_index('date', inplace=True)
    disease_data.set_index('date', inplace=True)
    
    # Get common date range
    start_date = gt_data.index.min()
    end_date = gt_data.index.max()
    
    print(f"\nAnalysis for {data_type} data:")
    print(f"Start date: {start_date.strftime('%Y-%m-%d')}")
    print(f"End date: {end_date.strftime('%Y-%m-%d')}")
    
    # Filter disease data to GT date range
    disease_filtered = disease_data[start_date:end_date]
    
    # Get intersection of dates
    common_dates = gt_data.index.intersection(disease_filtered.index)
    
    print(f"Number of days analyzed: {len(common_dates)}")
    
    # Filter both datasets to common dates
    gt_filtered = gt_data.loc[common_dates]
    disease_filtered = disease_filtered.loc[common_dates]
    
    results = []
    case_column = f"{case_type}_cases"
    
    # Analyze each GT column
    for column in gt_filtered.columns:
        gt_values = gt_filtered[column].values
        disease_values = disease_filtered[case_column].values
        
        # Calculate Pearson correlation and p-value
        pearson_corr, p_value = stats.pearsonr(gt_values, disease_values)

        # Kendall's Tau correlation - using filtered disease_values
        kendall_corr, kendall_pval = stats.kendalltau(gt_values, disease_values)
        
        # Calculate R-squared and slope
        slope, intercept, r_value, _, std_err = stats.linregress(gt_values, disease_values)
        r_squared = r_value ** 2
        
        # Calculate Cohen's d and Hedges' g
        cohens_d = calculate_cohens_d(gt_values, disease_values)
        hedges_g = calculate_hedges_g(gt_values, disease_values)
        
        results.append({
            'tag': column,
            'pearson_correlation': pearson_corr,  # Added this line to include Pearson correlation
            'pearson_pvalue': p_value,
            'r_squared': r_squared,
            'slope': slope,
            'kendall_correlation': kendall_corr,
            'kendall_pvalue': kendall_pval,
            'cohens_d': cohens_d,
            'hedges_g': hedges_g
        })
    
    return results

In [16]:
# Read the data
gt_msv = pd.read_csv('../gt_preprocessed_data/gt_msv_stitched/3_gt_msv_stitched_compute.csv')
gt_rsv = pd.read_csv('../gt_preprocessed_data/gt_rsv_stitched/4_gt_normal_rescaled_rsv.csv')
confirmed_cases = pd.read_csv('disease_confirmed_daily_cases.csv')
active_cases = pd.read_csv('disease_active_cases.csv')

In [17]:
def get_descriptive_stats(data, date_range=None):
    if date_range is not None:
        # Convert dates to datetime
        data['date'] = pd.to_datetime(data['date'])
        start_date, end_date = date_range
        # Filter data to date range
        data = data[(data['date'] >= start_date) & (data['date'] <= end_date)]
    
    # Get values (skip date column)
    values = data.iloc[:, 1]
    
    # Calculate Jarque-Bera test
    jb_stat, jb_pvalue = stats.jarque_bera(values)
    
    # Calculate statistics
    stats_dict = {
        'count': len(values),
        'mean': np.mean(values),
        'median': np.median(values),
        'std': np.std(values),
        'min': np.min(values),
        'max': np.max(values),
        'skewness': stats.skew(values),
        'kurtosis': stats.kurtosis(values, fisher=True),  # Fisher kurtosis (excess kurtosis)
        'jarque_bera_stat': jb_stat,
        'jarque_bera_pvalue': jb_pvalue
    }
    return stats_dict

In [18]:
# Read the data
confirmed_cases = pd.read_csv('disease_confirmed_daily_cases.csv')
active_cases = pd.read_csv('disease_active_cases.csv')
gt_data = pd.read_csv('../gt_preprocessed_data/gt_msv_stitched/3_gt_msv_stitched_compute.csv')

# Get GT data date range
gt_data['date'] = pd.to_datetime(gt_data['date'])
date_range = (gt_data['date'].min(), gt_data['date'].max())

def print_stats(stats, title):
    print(f"\n{title}")
    print(f"Number of observations: {stats['count']}")
    print(f"Mean: {stats['mean']:.2f}")
    print(f"Median: {stats['median']:.2f}")
    print(f"Standard Deviation: {stats['std']:.2f}")
    print(f"Minimum: {stats['min']:.2f}")
    print(f"Maximum: {stats['max']:.2f}")
    print(f"Skewness: {stats['skewness']:.4f}")
    print(f"Kurtosis (excess): {stats['kurtosis']:.4f}")
    print(f"Jarque-Bera statistic: {stats['jarque_bera_stat']:.4f}")
    print(f"Jarque-Bera p-value: {stats['jarque_bera_pvalue']:.4e}")
    print(f"Normal distribution test: {'Rejected' if stats['jarque_bera_pvalue'] < 0.05 else 'Not rejected'} (alpha=0.05)")

# Calculate and print statistics for full dataset
confirmed_stats = get_descriptive_stats(confirmed_cases)
print_stats(confirmed_stats, "Descriptive Statistics for Confirmed Daily Cases (Full Dataset):")

active_stats = get_descriptive_stats(active_cases)
print_stats(active_stats, "Descriptive Statistics for Active Cases (Full Dataset):")

# Calculate and print statistics for aligned date range
confirmed_stats_aligned = get_descriptive_stats(confirmed_cases, date_range)
print_stats(confirmed_stats_aligned, 
           f"Descriptive Statistics for Confirmed Daily Cases (Aligned with GT data: {date_range[0].strftime('%Y-%m-%d')} to {date_range[1].strftime('%Y-%m-%d')}):")

active_stats_aligned = get_descriptive_stats(active_cases, date_range)
print_stats(active_stats_aligned,
           f"Descriptive Statistics for Active Cases (Aligned with GT data: {date_range[0].strftime('%Y-%m-%d')} to {date_range[1].strftime('%Y-%m-%d')}):")


Descriptive Statistics for Confirmed Daily Cases (Full Dataset):
Number of observations: 426
Mean: 1050.20
Median: 463.50
Standard Deviation: 1307.69
Minimum: 1.00
Maximum: 7591.00
Skewness: 1.9566
Kurtosis (excess): 3.6271
Jarque-Bera statistic: 505.3090
Jarque-Bera p-value: 1.8773e-110
Normal distribution test: Rejected (alpha=0.05)

Descriptive Statistics for Active Cases (Full Dataset):
Number of observations: 1140
Mean: 7374.85
Median: 2079.00
Standard Deviation: 13870.68
Minimum: 0.00
Maximum: 96379.00
Skewness: 3.4274
Kurtosis (excess): 13.7824
Jarque-Bera statistic: 11254.8502
Jarque-Bera p-value: 0.0000e+00
Normal distribution test: Rejected (alpha=0.05)

Descriptive Statistics for Confirmed Daily Cases (Aligned with GT data: 2020-03-16 to 2021-03-15):
Number of observations: 365
Mean: 666.95
Median: 405.00
Standard Deviation: 685.79
Minimum: 1.00
Maximum: 3944.00
Skewness: 1.8020
Kurtosis (excess): 3.1199
Jarque-Bera statistic: 345.5857
Jarque-Bera p-value: 9.0576e-76
Normal

In [19]:
# Normalize the disease cases
confirmed_cases['confirmed_cases'] = confirmed_cases['confirmed_cases'] / 100
active_cases['active_cases'] = active_cases['active_cases'] / 1000

In [20]:
print(active_cases['active_cases'])

0       0.002
1       0.013
2       0.020
3       0.029
4       0.030
        ...  
1135    0.377
1136    0.377
1137    0.377
1138    0.377
1139    0.000
Name: active_cases, Length: 1140, dtype: float64


In [21]:
print(confirmed_cases['confirmed_cases'])

0       0.02
1       0.11
2       0.07
3       0.10
4       0.02
       ...  
421    23.08
422    17.54
423    15.56
424    19.42
425    20.62
Name: confirmed_cases, Length: 426, dtype: float64


In [22]:
# Analyze correlations for MSV data
print("\nAnalyzing MSV data correlations with normalized confirmed cases (divided by 100):")
msv_confirmed_results = analyze_correlations(gt_msv, confirmed_cases, "confirmed", "MSV")
for result in msv_confirmed_results:
    print(f"\nTag: {result['tag']}")
    print(f"Pearson correlation: {result['pearson_correlation']:.10f}")
    print(f"P-value: {result['pearson_pvalue']:.10f}")
    print(f"R-squared: {result['r_squared']:.10f}")
    print(f"Slope: {result['slope']:.10f}")
    print(f"Kendall correlation: {result['kendall_correlation']:.10f}")
    print(f"P-value: {result['kendall_pvalue']:.10f}")
    print(f"Cohen's d: {result['cohens_d']:.10f}")
    print(f"Hedges' g: {result['hedges_g']:.10f}")

print("\nAnalyzing MSV data correlations with normalized active cases (divided by 1000):")
msv_active_results = analyze_correlations(gt_msv, active_cases, "active", "MSV")
for result in msv_active_results:
    print(f"\nTag: {result['tag']}")
    print(f"Pearson correlation: {result['pearson_correlation']:.10f}")
    print(f"P-value: {result['pearson_pvalue']:.10f}")
    print(f"R-squared: {result['r_squared']:.10f}")
    print(f"Slope: {result['slope']:.10f}")
    print(f"Kendall correlation: {result['kendall_correlation']:.10f}")
    print(f"P-value: {result['kendall_pvalue']:.10f}")
    print(f"Cohen's d: {result['cohens_d']:.10f}")
    print(f"Hedges' g: {result['hedges_g']:.10f}")

# Analyze correlations for RSV data
print("\nAnalyzing RSV data correlations with normalized confirmed cases (divided by 100):")
rsv_confirmed_results = analyze_correlations(gt_rsv, confirmed_cases, "confirmed", "RSV")
for result in rsv_confirmed_results:
    print(f"\nTag: {result['tag']}")
    print(f"Pearson correlation: {result['pearson_correlation']:.10f}")
    print(f"P-value: {result['pearson_pvalue']:.10f}")
    print(f"R-squared: {result['r_squared']:.10f}")
    print(f"Slope: {result['slope']:.10f}")
    print(f"Kendall correlation: {result['kendall_correlation']:.10f}")
    print(f"P-value: {result['kendall_pvalue']:.10f}")
    print(f"Cohen's d: {result['cohens_d']:.10f}")
    print(f"Hedges' g: {result['hedges_g']:.10f}")

print("\nAnalyzing RSV data correlations with normalized active cases (divided by 1000):")
rsv_active_results = analyze_correlations(gt_rsv, active_cases, "active", "RSV")
for result in rsv_active_results:
    print(f"\nTag: {result['tag']}")
    print(f"Pearson correlation: {result['pearson_correlation']:.10f}")
    print(f"P-value: {result['pearson_pvalue']:.10f}")
    print(f"R-squared: {result['r_squared']:.10f}")
    print(f"Slope: {result['slope']:.10f}")
    print(f"Kendall correlation: {result['kendall_correlation']:.10f}")
    print(f"P-value: {result['kendall_pvalue']:.10f}")
    print(f"Cohen's d: {result['cohens_d']:.10f}")
    print(f"Hedges' g: {result['hedges_g']:.10f}")


Analyzing MSV data correlations with normalized confirmed cases (divided by 100):

Analysis for MSV data:
Start date: 2020-03-16
End date: 2021-03-15
Number of days analyzed: 364

Tag: flu
Pearson correlation: -0.0341785857
P-value: 0.5156709395
R-squared: 0.0011681757
Slope: -0.0111745939
Kendall correlation: 0.0678856127
P-value: 0.0543252559
Cohen's d: 3.2959460359
Hedges' g: 3.2925399601

Tag: cough
Pearson correlation: 0.0860404107
P-value: 0.1012255830
R-squared: 0.0074029523
Slope: 0.0339413813
Kendall correlation: 0.1261305603
P-value: 0.0003527914
Cohen's d: 4.9513214056
Hedges' g: 4.9462046422

Tag: fever
Pearson correlation: 0.1110092028
P-value: 0.0342442646
R-squared: 0.0123230431
Slope: 0.0558071491
Kendall correlation: 0.1218228988
P-value: 0.0005617676
Cohen's d: 6.3498431281
Hedges' g: 6.3432811132

Tag: headache
Pearson correlation: 0.0469025050
P-value: 0.3722556894
R-squared: 0.0021998450
Slope: 0.0285704030
Kendall correlation: 0.0429262711
P-value: 0.2248177518
C